# Deploy a un endpoint online

## Conectar a workspace

In [4]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import ManagedOnlineEndpoint, Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes
import datetime

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

Found the config file in: ./config.json


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Conectado al Workspace: naturgyml


In [6]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

# online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

online_endpoint_name = 'my-predict-failure-endpoint-ces'

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow predict failure model",
    auth_mode="key",
)

Crear el endpoint:

In [3]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://my-predict-failure-endpoint-ces.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://my-predict-failure-endpoint-ces.eastus2.inference.ml.azure.com/swagger.json', 'name': 'my-predict-failure-endpoint-ces', 'description': 'Online endpoint for MLflow predict failure model', 'tags': {}, 'properties': {'createdBy': 'Carlos Esteve', 'createdAt': '2025-06-02T20:57:24.318490+0000', 'lastModifiedAt': '2025-06-02T20:57:24.318490+0000', 'azureml.onlineendpointid': '/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourcegroups/naturgy/providers/microsoft.machinelearningservices/workspaces/naturgyml/onlineendpoints/my-predict-failure-endpoint-ces', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/providers/Microsoft.MachineLearningServices/locations/eastus2/mfeOperationsStatus/oeidp:00c02b0e-cf52-402e-a813-c08

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> ¡IMPORTANTE!¡Esperar hasta que el endpoint se cree con éxito antes de continuar!
Una notificación verde debería aparecer en el estudio. </p>



## Configurar el deployment

### Modelo registrado

In [12]:
reg_model=ml_client.models.get(name="predict_failure_rf_model", version="1")
reg_model.id

'/subscriptions/7decb7a4-f615-4cc3-9d7d-5de10998373f/resourceGroups/naturgy/providers/Microsoft.MachineLearningServices/workspaces/naturgyml/models/predict_failure_rf_model/versions/1'

In [13]:
# definir un nuevo deployment con el entorno personalizado
green_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=online_endpoint_name,
    model=reg_model.id,
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
    environment="azureml:sklearn-azure-ai-env:4",
)

In [ ]:
# Crear el deployment
ml_client.online_deployments.begin_create_or_update(green_deployment).result()

Asignar el % de tráfico.

In [ ]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
# endpoint.traffic = {"green": 20}
ml_client.begin_create_or_update(endpoint).result()

## Probar el deploymnent

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample_data.json",
)

if response[1]=='1':
    print("No fallo")
else:
    print ("Fallo")

## Listar los endpoints

In [15]:
endpoints = ml_client.online_endpoints.list()
for endp in endpoints:
    print(endp.name)

my-predict-failure-endpoint-ces


## Obtener detalles de un endpoint

In [16]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'green': 0}
https://my-predict-failure-endpoint-ces.eastus2.inference.ml.azure.com/score


## Borrar el endpoint y deployment

In [17]:
# Eliminar un deployment
ml_client.online_deployments.begin_delete(
    name='green',
    endpoint_name=online_endpoint_name
).result()

In [ ]:
# Eliminar el endpoint
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

...